### **Azure OpenAI Chat**

In [55]:
from openai import AzureOpenAI
import configparser
import json

In [56]:
config = configparser.ConfigParser()
config.read('config.ini')
api_key = config.get('default', 'AZURE_OPENAI_KEY')
api_endpoint = config.get('default','AZURE_OPENAI_ENDPOINT')

In [33]:
client = AzureOpenAI(
  azure_endpoint = api_endpoint, 
  api_key=api_key,  
  api_version="2024-02-15-preview"
)

In [ ]:

user_prompt = "¿Qué selección ganó la copa mundial de futbol en 2018?"

system_prompt = """Responde las preguntas de la manera más veraz posible.
                Si no sabes la respuesta contesta: "No lo sé"

Por ejemplo:

Pregunta de Usuario: ¿En qué meses se desarrolla el verano en el hemisferio norte?
Respuesta: junio, julio y agosto

"""
message_text = [{"role":"system","content":system_prompt},
                {"role":"user","content":user_prompt}]

response = client.chat.completions.create(
  model= "gpt-4",
  messages = message_text,
  temperature = 0.3
)

response.choices[0].message.content

In [9]:
system_prompt = '''Imagina que eres un miembro del equipo de marketing de una empresa de tecnología. Eres experto en campañas y engagement en redes sociales'''

user_prompt = '''Crea 3 hashtags para las campaña en redes sociales este 8 de Marzo, relacionados con las muejeres tecnólogas

De este formato:
Hashtag 1:
Hashtag 2:
Hashtag 3:
'''

In [ ]:
message_text = [{"role":"system","content":system_prompt},
                {"role":"user","content":user_prompt}]

response = client.chat.completions.create(
  model= "gpt-4",
  messages = message_text,
  temperature = 1
)



In [ ]:
print (response.choices[0].message.content)

### **Chat con tus Datos**

In [57]:
search_endpoint = config.get('default', 'AZURE_SEARCH_ENDPOINT')
search_key = config.get('default', 'AZURE_SEARCH_KEY')
search_index_name = "lil-demo-ix"

In [58]:
azure_oai_deployment = "gpt-4"

client = AzureOpenAI(
    base_url=f"{api_endpoint}/openai/deployments/{azure_oai_deployment}/extensions",
    api_key=api_key,
    api_version="2023-09-01-preview")

In [59]:
text = input('\nEnter a question:\n')

In [60]:
print("Request: " + text + "\n")

Request: ¿Por qué la reina quería matar a Blanca Nieves?



In [61]:
extension_config = dict(dataSources = [  
        { 
            "type": "AzureCognitiveSearch", 
            "parameters": { 
                "endpoint":search_endpoint, 
                "key": search_key, 
                "indexName": search_index_name,
            }
        }]
        )

In [ ]:
response = client.chat.completions.create(
            model = azure_oai_deployment,
            temperature = 0.5,
            max_tokens = 1000,
            messages = [
                {"role": "system", "content": "You are a helpful assistant who knows about children stories"},
                {"role": "user", "content": text}
            ],
            extra_body = extension_config
        )

In [54]:

print("Response: " + response.choices[0].message.content + "\n")

print("\nContext information:\n")
context = response.choices[0].message.context
for context_message in context["messages"]:
    context_json = json.loads(context_message["content"])
    print(json.dumps(context_json, indent=2))

Response: La Reina, que era la madrastra de Blanca Nieves, estaba consumida por la envidia debido a la belleza de Blanca Nieves. Temía perder su posición como la más bella del reino, por lo que ordenó a un cazador que llevara a Blanca Nieves al bosque y la matara [doc1].


Context information:

{
  "citations": [
    {
      "content": "Hab\u00eda una vez, en un reino muy lejano, una hermosa princesa llamada Blanca Nieves. Ella era tan hermosa que su madrastra, la Reina, se consum\u00eda por la envidia, deseando ser la m\u00e1s bella del reino. La Reina, temiendo perder su posici\u00f3n, orden\u00f3 a un cazador que llevara a Blanca Nieves al bosque y la matara.\r\n\r\nSin embargo, el coraz\u00f3n del cazador se llen\u00f3 de compasi\u00f3n por la dulce Blanca Nieves, y en lugar de hacerle da\u00f1o, la dej\u00f3 en libertad en el bosque. Blanca Nieves, asustada pero valiente, se adentr\u00f3 en el espeso bosque y encontr\u00f3 refugio en una peque\u00f1a caba\u00f1a, propiedad de siet